In [149]:
import csv
import re
import pandas as pd
import os
import sys
import argparse



d = {'Sample_ID': [1, 2],"index": ["asd","asdsa"], 'Sample_Project': ["", ""]}

df = pd.DataFrame(data=d)
df
header_projectid='TEST_PROJECT'

if 'Sample_Project' not in df.columns:
    print(f' ... ... Warning:  Sample_Project colum not found in [Data]' )
    print(f' ... ... ... Forcing all Samples in [Data] Sample_Project to ProjectID from header :  {header_projectid}' )
    df['Sample_Project']=header_projectid
elif len(df['Sample_Project'].unique())==1:
    if not df['Sample_Project'].unique():
        print(f' ... ... Warning:  Sample_Project colum contains only blank values' )
        print(f' ... ... ... Forcing all Samples in [Data] Sample_Project to ProjectID from header :  {header_projectid}' )
        df['Sample_Project']=header_projectid

df

if header_projectid=='TEST_PROJECT' and header_projectid in ["TEST_PROJECT",""]:
    print('YES!!!')



 ... ... Warning:  Sample_Project colum contains only blank values
 ... ... ... Forcing all Samples in [Data] Sample_Project to ProjectID from header :  TEST_PROJECT
YES!!!


In [150]:
import pandas as pd

d = {'Sample_ID': [],"index": [], 'Sample_Project': []}

df = pd.DataFrame(data=d)
df
df.shape[0]
if df.shape[0] == 0:
    raise ValueError(f'Error: No [Data] rows are found in sheet. Demux cannot be performed. Only rawdata pipelineProfiles accept empty [Data] sections' )



ValueError: Error: No [Data] rows are found in sheet. Demux cannot be performed. Only rawdata pipelineProfiles accept empty [Data] sections

In [ ]:
import csv
import re
import pandas as pd
import os
import sys
import argparse

## extract param function. Return value from second instance if found. else return blank
def get_param(param_name=None, myDict=None):
    if param_name in myDict.keys(): return(myDict[param_name][1])
    else: return('')
## end function


## -------------------------------
##  Params uncomment when debug on local
## -------------------------------
os.chdir('/Users/david/tmp/') ## use if local
sheet_name = 'CTG_SampleSheet.labsheet.test.csv' ## use if local

# Get batchid from labsheet name - add to demux-samplesheet
# batchid_sheetname=sheet_name.split(".")[2]

# # The Sample_Project, Sample_ID, and Sample_Name columns accept alphanumeric characters, hyphens (-), and underscores (_).
force_Sample_Name = True  # if to force Sample_Name(s) supplied in [Data] column to the same as Sample_ID
fastq_suffix = "_001.fastq.gz" # "Suffix needed to auto-generate fastq file names generated by bcl2fastq. If NULL no bam file names will be genrerated"
bam_suffix = "_Aligned.sortedByCoord.out.bam"  ## "Suffix needed to auto generate bam file names (typically generated by STAR). If NULL no bam file names will be genrerated"
force_fastq_names = False # Set to true if topai overwrite fastq filenames. By defualt (fastq_1/fastq_2) columns will not be overwritten if present (even though fastq_suffix is supplied)
force_bam_names = False # Set to true if to overwrite bam filenames. By defualt (bam) column will not be overwritten if present (even though bam_suffix is supplied)

## ADD UNIQUE FASTQ IF MULTIPLE LANES * collapse lanes * Special cases when same sample is distributed over multiple lanes within a single project.
allow_dups_over_lanes = True # If to allow duplicates (within one project) on multiple lanes. Rare on NovaSeq but can be found for S4 with lane divider. One sample may be run on both lane 1/2 or on 3/4.
collapse_lanes = True ## Like allow_dups_over_lanes, affects project specific samplsheets NOT demux sheet.  special cases - when a single (same) sample is present in multiple lanes AND --noLaneSplitting is True in bcl2fastq. Then SampleSheet should be collapsed from Lane to individual sample (fastq R1/R2 files )
#force_fastq_names = False

cwd = os.path.basename(os.getcwd())
runfolder_root="/projects/fs1/nas-sync/upload/"



## Pipeline dict. check allowed Pipeline & pipeline profiles
pipelineDict = {
    'seqonly': ['bcl2fastq_default','fastq_demux','rawdata_runfolder'],
    'ctg-rnaseq': ['rnaseq_mrna','rnaseq_total','uroscan','fastq_demux','rawdata_runfolder','rawdata'],
    'dna-dragen': ['panel_twist_comprehensive_dragen','panel_gmck_dragen','panel_gms_dragen','bam_alignment_dragen','wgs_dragen'],
    'demux-runfolder': ['bcl2fastq_default']
    }



## a dictionary is used to find the corresponding [Data] section to a [Header] param
## key = [Header] param name
##  DataCol: [Data] column name
##  Catenate: boolean if to collapse multiple entries. This is id Data column contanins multiple (non unique) values, if to collapse these in Header section with semicolon.
##  RegExp: regexp to parse. what characters are allowed for this entry. Leave blank if to use the default character setup set in default_regexp
##  Controlled: if the entry has a controlled vocab or not (not yet implemented)
default_regexp='[^0-9a-zA-Z\_\.\-\+\@\(\)\;\,\'\"\| ]+'

params_dict = {
    'ProjectID': {'DataCol': 'Sample_Project','Catenate': True,'RegExp': '[^0-9a-zA-Z\_\|]+','Controlled': False},
    'PipelineName': {'DataCol': 'PipelineName','Catenate': False,'RegExp': '','Controlled': True},
    'PipelineVersion': {'DataCol': 'PipelineVersion','Catenate': False,'RegExp': '','Controlled': True},
    'PipelineProfile': {'DataCol': 'PipelineProfile','Catenate': False,'RegExp': '','Controlled': True},
    'Species': {'DataCol': 'Sample_Species','Catenate': False,'RegExp': '','Controlled':False},
    'ReferenceGenome': {'DataCol': 'Sample_ReferenceGenome','Catenate': False,'RegExp': '','Controlled':False},
    'email-ctg-lab': {'DataCol': 'email_ctg_lab','Catenate': False,'RegExp': '[^0-9a-zA-Z\.\-\_\@\|]+','Controlled': False},
    'email-ctg-bnf': {'DataCol': 'email_ctg_bnf','Catenate': False,'RegExp': '[^0-9a-zA-Z\.\-\_\@\|]+','Controlled': False},
    'email-ctg-all': {'DataCol': 'email_ctg_all','Catenate': False,'RegExp': '[^0-9a-zA-Z\.\-\_\@\|]+','Controlled': False},
    'name-pi': {'DataCol': 'name_pi','Catenate': False,'RegExp': '','Controlled':False},
    'email-customer': {'DataCol': 'email_customer','Catenate': False,'RegExp': '[^0-9a-zA-Z\.\-\_\@\|]+','Controlled': False},
    'Assay': {'DataCol': 'Assay','Catenate': False,'RegExp': '','Controlled': True},
    'IndexAdapters': {'DataCol': 'IndexAdapters','Catenate': False,'RegExp': '','Controlled': True},
    'Strandness': {'DataCol': 'Sample_Strandness','Catenate': False,'RegExp': '','Controlled': True},
    'FragmentationTime': {'DataCol': 'fragmentation_time','Catenate': False,'RegExp': '','Controlled': True},
    'PCR-cycles': {'DataCol': 'pcr_cycles','Catenate': False,'RegExp': '','Controlled': True},
    'PairedEnd': {'DataCol': 'Sample_PairedEnd','Catenate': False,'RegExp': '','Controlled': True},
    'PoolConcNovaSeq': {'DataCol': 'Pool_Conc_NovaSeq','Catenate': False,'RegExp': '','Controlled': True},
    'PoolMolarityNovaSeq': {'DataCol': 'Pool_Molarity_NovaSeq','Catenate': False,'RegExp': '','Controlled': True}}

params_datacols=[params_dict[c]['DataCol'] for c in params_dict.keys()]
params_datacols=dict.fromkeys(params_datacols)
for c in params_dict.keys():
    params_datacols[params_dict[c]['DataCol']]=c

print(params_datacols)




def harmonize_header_params(input_row=None, data_mat=None, data_col=None, allowMultiple=None, ingoreBlanks=None):
    ## function for harmonizing parameters that are present in [Header] and [Data] (individual samples)
    ## [Header] and [Data] param pairs often do not have identical names
    ## Main principle is to look at values in [Data] column and replace the [Header] with that value(s)
    ##  - if unique value in [Data] - Replace!
    ##  - if >1 value collapse 'multiple' (default), or separate values by comma.
    return_row = input_row
    if data_col in data_mat.columns.tolist():
        if len(data_mat[data_col].unique())== 1:
            return_row[1] = data_mat[data_col].tolist()[0]
        else:
            return_row[1] = 'multiple'
            if allowMultiple==False:
                raise ValueError(f'Error: Multiple values found in [Data] column "{data_col}" when harmonizing [Header] and [Data] params. Multiple values are not allowed within one and the same project as defined by the "params_dict" object in this python script. Values found were:  {data_mat[data_col].unique()}' )
        if not return_row[1]==input_row[1]:
            print(f' ... ... Harmonizing values. [Header] param "{input_row[0]}" changed from "{input_row[1]}" to [Data] "{data_col}" columns value: {return_row[1]}')
        # if return_row[1]==input_row[1]: ## no action

    return(return_row)
    ## end function




{'Sample_Project': 'ProjectID', 'PipelineName': 'PipelineName', 'PipelineVersion': 'PipelineVersion', 'PipelineProfile': 'PipelineProfile', 'Sample_Species': 'Species', 'Sample_ReferenceGenome': 'ReferenceGenome', 'email_ctg_lab': 'email-ctg-lab', 'email_ctg_bnf': 'email-ctg-bnf', 'email_ctg_all': 'email-ctg-all', 'name_pi': 'name-pi', 'email_customer': 'email-customer', 'Assay': 'Assay', 'IndexAdapters': 'IndexAdapters', 'Sample_Strandness': 'Strandness', 'fragmentation_time': 'FragmentationTime', 'pcr_cycles': 'PCR-cycles', 'Sample_PairedEnd': 'PairedEnd', 'Pool_Conc_NovaSeq': 'PoolConcNovaSeq', 'Pool_Molarity_NovaSeq': 'PoolMolarityNovaSeq'}


In [ ]:
def find_csv_delimiter(sheet_name=None):
    ## function to determine if csv file uses comma or semicolon as separator
    ## simply counts the number of ',' and ';'. Who wins this battle will be thew winner
    ## Input: csv file path
    ## Output: a character - separator (',' or ';')

  #sheet_name='/Users/david/tmp/CTG_SampleSheet.labsheet.test.csv'
  print(f' ... determining csv file separator')
  print(f' ... ... Reading: {sheet_name} ')

  count_comma = 0
  count_semic = 0
  csvfile = open(sheet_name, "r")
  for i in csvfile:
    for c in i:
      if c == ',': count_comma += 1
      elif c == ';': count_semic += 1
  csvfile.close()
  print(f' ... ... {count_comma} commas vs  {count_semic} semicolons')
  if count_comma >= count_semic:
    return_char=','
  else:
    return_char=','
  print(f' ... ... returning "{return_char}" ' )
  return(return_char)
      

In [ ]:
def replace_characters_foo(list_in=None, RegExp='[^0-9a-zA-Z\_\.\-\+\@\(\)\;\,\'\"\| ]+', my_sub=''):
    ## function to replace non-allowed characters with a character. 
    ## input is a list. loops thorugh the entire list
    p = re.compile(RegExp)
    list_out = list_in
    l_index = 0
    for list_i in list_in:
        substring=p.sub(my_sub, list_i)    
        if not substring == list_i:
            print(f' ... ... ... Illegal character in "{list_i}". replacing with "{substring}"')
            list_out[l_index]=substring
        l_index+=1
    # print(f'{list_out}')   
    return(list_out)

def fix_booleans_foo(list_in):
    ## replace booleans with lowercase (fit for bash/nextflow scripting)
    list_out = list_in
    l_index = 0
    for list_i in list_in:
        if list_i.lower() in ['true','false']:
            list_out[l_index]=list_i.lower()
        l_index+=1
    return(list_out)

myrow=['', '9941_19_N_e1', '9941_19_N_e1', '2022_017_f1', 'dna-batch-003', 'fastq_only', '', 'A04', 'UDI0025', 'ACTAAGAT', 'UDI0025', 'CCGCGGTT', 'FC1', 'SeqPool_1', '', 'Homosapiens', '', '', '', 'Normal', '30', '33.5', '', '1842.5', '2022-017', 'A01', '797', '2.78']
row = replace_characters_foo(list_in=myrow, RegExp='[^0-9a-zA-Z\_+]', my_sub='') ## replace all (including illegal spaces) with regular space



 ... ... ... Illegal character in "dna-batch-003". replacing with "dnabatch003"
 ... ... ... Illegal character in "33.5". replacing with "335"
 ... ... ... Illegal character in "1842.5". replacing with "18425"
 ... ... ... Illegal character in "2022-017". replacing with "2022017"
 ... ... ... Illegal character in "2.78". replacing with "278"


In [ ]:


def read_section(sheet_name=None, sheet_section=None, section_is_dataframe=False, allowMultiple=False, RegExp='[^0-9a-zA-Z\_\.\-\+\@\(\)\;\,\'\"\| ]+'):
    ## function for reading the different sections in a IEM style sample sheet.
    ## Each section is defined with a header brackets, such as [Header], [Data] etc
    ## Input: a CTG style samplesheet
    ##       accepts both comma and semicolon separated (determined using find_csv_delimiter function)
    ## Output: a samplesheet section that is parsed/curated, to use/print in a parsed sampleheet by the ctg-parse-samplesheeet script
    ## Behaviour: 
    ##   duplicate params/columns within a section is not allowed.
    ##   params with (all) blank values are removed
    ##   blank rows will always mark the end of a section, i.e. a section starts from its [Data] and ends at 1st blank row

    ## Arguments:
    ##   sheet_name: name of csv sample sheet
    ##   sheet_section: section to parse and extract. Use brackets!!  
    ##   section_type: argument (rows or matrix) will define how the section is read and returned
    ##   RegExp: Allowed characters. A default regular expression to control allowed characters within this section. Here, the regexp is pretty inclusive, listing characters allowed over all sectionss. For more stringent regexp filtering add these steps later 
    ## 
    import csv
    import re
    import pandas as pd
    import os
    import sys
    import argparse

    ## debugging:
    # sheet_name='/Users/david/tmp/CTG_SampleSheet.labsheet.test.csv'
    # section_is_dataframe=True
    # sheet_section='[Data]'
    # RegExp='[^0-9a-zA-Z\_\.\-\+\@\(\)\;\,\'\"\| ]+'

    ## Start function
    ## --------------
    n_rows=len(open(sheet_name).readlines())
    print(f' ... Fetching {sheet_section} section from "{sheet_name}"')
    print(f' ... ... ')
    sheet_delim = find_csv_delimiter(sheet_name=sheet_name) # determine delimiter
    

    with open(sheet_name, 'r', encoding='utf-8-sig') as csvfile:
        allines = csv.reader(csvfile, delimiter=(sheet_delim), quotechar='"', skipinitialspace=True)    
        myLine = 0
        s_index = 0
        read_section = False
        myDict={} ## dictionary in which to store section to be read

        ## main approach is to read until between given section header and 1st blank line
        ## -------------------------------
        print(f' ... ... Reading lines in SampleSheet ...')
        print(f' ... ... (File is {n_rows} rows )')
        for row in allines:
            myLine+=1
            if len(row) < 2: # quickfix for if csv file has no proper commas. causes problems if only the param listed but not followed by comma (and a value)
                row.append('') # Append a blank value to the 'row' object (minimum length is 2)
            
            ## Read rows that span between section of interrest and first blank row after
            if row[0] == sheet_section:
                read_section = True
                found_section = True
                print(f' .. ... found supplied sheet_section header: {sheet_section} at line {myLine}')
                print(f' ... ... ... reading data')
                continue ## section header identified, continue reading next line and store (until blank line)
            elif read_section == True and all(elem == '' for elem in row):
                read_section = False
                print(f' ... ... stopped reding at blank line: {myLine}')
                continue
            
            ## If inbetween [section] and blank row
            if read_section == True: ## parse this line and store in output dict
                
                # replace very illegal characters using replace_characters_foo
                row = replace_characters_foo(list_in=row, RegExp='[\,]', my_sub=' ') ## replacing commas if present
                row = replace_characters_foo(list_in=row, RegExp='[\s]+', my_sub=' ') ## replace all (including illegal spaces) with regular space
                row = replace_characters_foo(list_in=row, RegExp=RegExp, my_sub='') ## replace all (including illegal spaces) with regular space
                
                # set any booleans to lowercase (true/false)
                row = fix_booleans_foo(row)

                if not section_is_dataframe:
                    ## If this section is not a matrix/data frame. Then expext one value per parameter (row)
                    ## <parameter>,<value>, ... i.e. row[0] is the parameter and row[1] is the value
                    ## store the row[1] in the output dict key row[0]. Raise error & exit if param already has been read
                    if row[0] in myDict.keys():
                        raise ValueError(f' ... ... ... Error: Duplicate {sheet_section} parameter found: "{row[0]}"')                
                    ## Store parameter value in myDict dictionary (dict key is the paramter) 
                    myDict[row[0]] = row[1]
                    print(f' ... ... ... read param:  {row[0]}')

                elif section_is_dataframe:
                    ## If data frame type of section. first import into dictionary (use s_index as row index)                
                    ## First row will become header - do not allow any special characters other than underscore here
                    if s_index == 0:
                        print(f' ... ... ... reading header row for data frame. ')
                        print(f' ... ... ... ... allow ony underscore as special character')
                        row = replace_characters_foo(list_in=row, RegExp='[^0-9a-zA-Z\_+]', my_sub='') ## replace all (including illegal spaces) with regular space
                        print(row)
                    myDict[s_index] = row
                    # print(f' ... ... ... read data frame row:  {s_index}')
            s_index += 1
        print(f' ... ... ok ')
        
        ## All data is read. Now crunch dictionary into panda data frame if section is df
        if section_is_dataframe and found_section:
            # generate Pandas Data Frame 
            print(f' ... ... section_is_dataframe set to true:')
            print(f' ... ... ... generating pandas dataframe')
            df = pd.DataFrame(myDict)
            df = df.transpose() # transpose from dict
            df.rename(columns=df.iloc[0], inplace=True) # first [Data] row is headers
            df = df.iloc[1: , :]
            print(f' ... returning data frame with dimensions: {df.shape}')
            return(df)
        elif section_is_dataframe and found_section:
            print(f' ... returning dictionary')
            return(myDict)
        else:
            print(f' ... section header {sheet_section} not found. returning ""')
            return('')

            
    

In [ ]:


def harmonize_header_params(param_name=None, input_val=None, data_mat=None, data_col=None, allowMultiple=None):
    ## function for harmonizing parameters that are present in [Header] with [Data] section entries (individual samples)
    ## [Header] and [Data] param pairs often do not have identical names
    ## Main principle is to look at values in [Data] column and replace the [Header] param with that value(s)
    ##  - if unique value in [Data] - Replace!
    ##  - if >1 value collapse 'multiple' (default)
    return_val = input_val
    if data_col in data_mat.columns.tolist():
        if len(data_mat[data_col].unique())== 1:
            return_val = data_mat[data_col].tolist()[0]
        else:
            return_val = 'multiple'
            if allowMultiple==False:
                raise ValueError(f'Error: Multiple values found in [Data] column "{data_col}" when harmonizing [Header] and [Data] params. Multiple values are not allowed within one and the same project as defined by the "params_dict" object in this python script. Values found were:  {data_mat[data_col].unique()}' )
        if not return_val==input_val:
            print(f' ... ... Harmonizing values. [Header] param "{param_name}" changed from "{input_val}" to [Data] "{data_col}" columns value: {return_val}')
        # if return_val[1]==input_row[1]: ## no action

    return(return_val)
    ## end function



In [ ]:
def read_runparameters(xmlfile=None):
    ## input: RunParameters.xml file
    ## output: dictionary with the juiciest info from a illumina RunParameters xml file. 
    ##   to be added in a CTG SampleSheet

    import csv
    import xml.etree.ElementTree as ET
    # xmlfile='/Users/david/scripts/ctg-parse-samplesheet/examples/RunParameters-Visium.xml'
    tree = ET.parse(xmlfile)
    # root = tree.getroot()
    
    params_dict={
        'FlowCellSerialBarcode':'',
        'ReadType':'',
        'Read1NumberOfCycles':'',
        'IndexRead1NumberOfCycles':'',
        'IndexRead2NumberOfCycles':'',
        'Read2NumberOfCycles':'',
        'Read2NumberOfCycles':'',
        'FlowCellStartDate':'',
        'FlowCellMode':'',
        'Side':'',
        'RunNumber':''}

    for paramname in params_dict:    
        params_dict[paramname]=''.join(tree.findall(f'.//{paramname}')[0].itertext())
        print(f' ... ... ... {paramname}: {params_dict[paramname]}')
    
    return(params_dict)

read_runparameters(xmlfile='/Users/david/scripts/ctg-parse-samplesheet/examples/RunParameters-Visium.xml')


 ... ... ... FlowCellSerialBarcode: HFNN5DRXY
 ... ... ... ReadType: PairedEnd
 ... ... ... Read1NumberOfCycles: 28
 ... ... ... IndexRead1NumberOfCycles: 10
 ... ... ... IndexRead2NumberOfCycles: 10
 ... ... ... Read2NumberOfCycles: 90
 ... ... ... FlowCellStartDate: 09/14/2021 16:20:00
 ... ... ... FlowCellMode: SP
 ... ... ... Side: A
 ... ... ... RunNumber: 464


{'FlowCellSerialBarcode': 'HFNN5DRXY',
 'ReadType': 'PairedEnd',
 'Read1NumberOfCycles': '28',
 'IndexRead1NumberOfCycles': '10',
 'IndexRead2NumberOfCycles': '10',
 'Read2NumberOfCycles': '90',
 'FlowCellStartDate': '09/14/2021 16:20:00',
 'FlowCellMode': 'SP',
 'Side': 'A',
 'RunNumber': '464'}

In [ ]:
sectionDict={}

## Import sample sheet sections
## ----------------------------
print(f' ... reading sample sheet sections')
sectionDict['Header']=read_section(sheet_name=sheet_name, sheet_section='[Header]', section_is_dataframe=False)
sectionDict['Projects']=read_section(sheet_name=sheet_name, sheet_section='[Projects]', section_is_dataframe=True)
sectionDict['Pools']=read_section(sheet_name=sheet_name, sheet_section='[Pools]', section_is_dataframe=True)
sectionDict['Data']=read_section(sheet_name=sheet_name, sheet_section='[Data]', section_is_dataframe=True)


## Curate & polish -- clean up some extra illegal characters ...
## -----------------------------


## Import RunParameters from xml (sectionDict['RunParameters'])
## -------------------------------------------------------
# also check if FlowCell serial number mathcs the one in RunParameters
import_run_params=True
if import_run_params:
    print(f' ... "import_run_params" is True: reading RunParameters.xml file')
    cwd_abs=os.path.abspath(os.getcwd())
    xmlfile=f'{cwd_abs}/RunParameters.xml'
    sectionDict['RunParameters']=read_runparameters(xmlfile)
    ## Check that FlowCell serial number mathcs the one in RunParameters
    ## if so get the full runfolder name
    print(f' ... ... checking serial FlowCell number')
    if not 'FlowCellSerialBarcode' in sectionDict['Header'].keys():
        raise ValueError(f'Error: FlowCellSerialBarcode is required in [Header] section.')
    if sectionDict['Header']['FlowCellSerialBarcode']=='': 
        raise ValueError(f'Error: FlowCellSerialBarcode must have a value in [Header] section.')
    if sectionDict['Header']['FlowCellSerialBarcode'] != sectionDict['RunParameters']['FlowCellSerialBarcode']:
        raise ValueError(f'Error: FlowCellSerialBarcode in RunParameters.xml and [Header] do NOT match: {sectionDict["RunParameters"]["FlowCellSerialBarcode"]} vs {sectionDict["Header"]["FlowCellSerialBarcode"]}' )
    
    # remove (duplicate) FlowCellSerialBarcode key from RunParameters
    del sectionDict['RunParameters']['FlowCellSerialBarcode']
    ## if OK now add RunFolder to Header sect (name of illumina Runfolder, i.e. current dir, cwd)
    sectionDict['Header']['RunFolder']=cwd

    #print(sectionDict['Header']['RunFolder'])
    #print(sectionDict['RunParameters']['FlowCellSerialBarcode'])
    print(f' ... ok ... ')



## Harmonize [Data] & [Header] params. 
# -------------------------------------
# Do this before checking requried params ... 
## harmonize_header_params function
# If param found in params_dict use harmonize_header_params function to replace [Header] value (if needed). Note that all blank columns have allready been removed

# !! ADD regexp check
print(f' ... harmonizing [Header] parameters wtih sample [Data] entries')
for paramname in params_dict.keys():
    print(paramname)
    if paramname not in sectionDict['Header']:
        sectionDict['Header'][paramname]=''
    print(sectionDict['Header'][paramname])
    sectionDict['Header'][paramname]=harmonize_header_params(param_name=paramname, 
        input_val=sectionDict['Header'][paramname], 
        data_mat=sectionDict['Data'],
        data_col=params_dict[paramname]['DataCol'])

print(f' ... harmonizing [Data] parameters')
## Force Sample_Project to [Header] ProjectID -- only if missing & not "multiple"



## Check required params & data cols
## -------------------------------
print(f' ... ')
print(f' ... Checking Required parameters')
required_params = {
    'Header':['ProjectID','PipelineName'],
    'Data':['Sample_ID','Sample_Project',]
}




if get_param('PipelineName',sectionDict['Header'])=='':
     print('NO')


## Required parameters are: 
##   ProjectID - or 

# # # Read & check Pipeline & Profile. (check allowed values when writing project specific sheet)
# # # =========================================================================
# print(f' ... Checking Pipeline & Profile in [Header]')
# name_found=False  ## PipelineName is required
# profile_found=False ## PipelineProfile is required
# for row in sectionDict['[Header]']:
#     if row == 'PipelineName':
#         header_pipelinename = sectionDict['[Header]'][row][1]
#         print(f' ... ... PipelineName: {header_pipelinename}')
#         name_found=True
#         # if not header_pipelinename in pipelineDict.keys():
#         #     raise ValueError(f'[Header] param "PipelineName" incorrectly specified. Must be one of {pipelineDict.keys()}' )
#     if row == 'PipelineProfile':
#         header_pipelineprofile = sectionDict['[Header]'][row][1]
#         print(f' ... ... PipelineProfile: {header_pipelineprofile}')
#         profile_found=True
#         # if not header_pipelineprofile in pipelineDict[header_pipelinename]:
#         #     raise ValueError(f'[Header] param "PipelineProfile" incorrectly specified. Must be one of {pipelineDict[header_pipelinename]}' )
# if not name_found: raise ValueError('[Header] param "PipelineName" must be specified' )
# if not profile_found: raise ValueError('[Header] param "PipelineProfile" must be specified' )
# print(f' ... ... ok')


## Check [Reads] vs [RunParameters]

## Fix Adapters!!!
## Check if in [data] section. Replace or append. append multiple how?
# Start from .... 


 ... reading sample sheet sections
 ... Fetching [Header] section from "CTG_SampleSheet.labsheet.test.csv"
 ... determining csv file separator
 ... ... Reading: /Users/david/tmp/CTG_SampleSheet.labsheet.test.csv 
 ... ... 1539 commas vs  0 semicolons
 ... ... returning "," 
 ... ... Reading lines in SampleSheet ...
 .. ... found supplied sheet_section header: [Header] at line 1
 ... ... ... reading data
 ... ... ... read param:  SheetType
 ... ... ... read param:  lab-batch-id
 ... ... ... read param:  ProjectID
 ... ... ... read param:  NumberSamples
 ... ... ... read param:  FlowCell_id
 ... ... ... read param:  FlowCellSerialBarcode
 ... ... ... read param:  FileName
 ... ... ... read param:  WorkflowType
 ... ... ... read param:  bcl2fastqArg
 ... ... ... read param:  Sequencing_Pool
 ... ... ... read param:  InstrumentType
 ... ... ... read param:  FlowCellType
 ... ... ... read param:  SharedFlowCell
 ... ... ... read param:  LaneDivider
 ... ... stopped reding at blank line: 16


57